<a id="toc"></a>
# Ray tracing plots with barc4plots
***

Contents :
1. [G1600 monochromator](#monoG1600)

In [ ]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'GPL-3.0'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '22/OCT/2024'
__changed__ = '22/OCT/2024'

import sys
import numpy as np
from numpy.polynomial import Polynomial
from barc4plots.barc4plots import PlotManager
import barc4xoc.aux_pyoptix as apx
import barc4xoc.aux_shadow as asw
from array import array
from scipy.optimize import minimize
from glob import glob
# checking we are using the correct python:
print(sys.executable)
print(sys.version)
# %matplotlib widget
import matplotlib.cm as cm


### Auxiliary functions

In [9]:
def plot_phase_space(beam, direction, prange=None, file_name=None):
    if prange is None:
        prange = [None, None, None, None]
    d = direction.capitalize()
    fig = PlotManager(axis_x=beam[f"{d}"]*1E6, axis_y=beam[f"{d}p"]*1E6)
    fig.additional_info('', f"${d.casefold()}$ [µm]", f"${d.casefold()}_p$ [µrad]", xmin=prange[0], xmax=prange[1], ymin=prange[2], ymax=prange[3])
    fig.aesthetics(dpi=600, LaTex=True, AspectRatio=False, FontsSize=None, grid=True, nbins=None)
    fig.info_scatter(ColorScheme=cm.magma, LineStyle='.', alpha=1, s=1, edgeColors='face', monochrome=False)
    fig.plot_scatter_hist(file_name)

def plot_beam_size(beam, prange, file_name=None):
    if prange is None:
        prange = [None, None, None, None]
    fig = PlotManager(axis_x=beam["X"]*1E6, axis_y=beam["Y"]*1E6)
    fig.additional_info('', '$x$ [µm]', '$y$ [µm]', xmin=prange[0], xmax=prange[1], ymin=prange[2], ymax=prange[3])
    fig.aesthetics(dpi=600, LaTex=True, AspectRatio=True, FontsSize=None, grid=True, nbins=None)
    fig.info_scatter(ColorScheme=2, LineStyle='.', alpha=1, s=1, edgeColors='face', monochrome=False)
    fig.plot_scatter_hist(file_name)

def plot_beam_div(beam, prange, file_name=None):
    if prange is None:
        prange = [None, None, None, None]
    fig = PlotManager(axis_x=beam["Xp"]*1E6, axis_y=beam["Yp"]*1E6)
    fig.additional_info('', '$x_p$ [µrad]', '$y_p$ [µrad]', xmin=prange[0], xmax=prange[1], ymin=prange[2], ymax=prange[3])
    fig.aesthetics(dpi=600, LaTex=True, AspectRatio=True, FontsSize=None, grid=True, nbins=None)
    fig.info_scatter(ColorScheme=2, LineStyle='.', alpha=1, s=1, edgeColors='face', monochrome=False)
    fig.plot_scatter_hist(file_name)


def get_beam_stats(beam, direction="both"):

    def objective_function(distance, spots, axis):
        return (spots[axis.capitalize()] + distance * spots[axis.capitalize()+"p"]).std()
    
    results = []

    if direction in ['x', 'both']:
        result_x = minimize(objective_function, 0, args=(beam, 'x'))
        results.append(result_x.x[0])
        print("Horizontal plane:")

        s = np.std(beam["X"])
        ds = np.std(beam["Xp"])
        msg = f">> RMS beam size {s*1E6:.1f} µm and divergence {ds*1E6:.1f} µrad"
        print(msg)
        print(f">> Beam focusing along X at {result_x.x[0]:.3f} m")

    if direction in ['y', 'both']:
        result_y = minimize(objective_function, 0, args=(beam, 'y'))
        results.append(result_y.x[0])

        s = np.std(beam["Y"])
        ds = np.std(beam["Yp"])
        print("Vertical plane:")
        msg = f">> RMS beam size {s*1E6:.1f} µm and divergence {ds*1E6:.1f} µrad"
        print(msg)
        print(f">> Beam focusing along Y at {result_y.x[0]:.3f} m")

    return results
    

def get_simulations_range(file_list, computer_code, **kwargs):

    nsigma = kwargs.get("nsigma", 3)
    nmax = kwargs.get("nmax", 1.05)

    X = []
    Y = []
    dX = []
    dY = []

    for simulation in file_list:
        if 'optix' in computer_code.lower():
            sim = apx.read_beam_from_csv(simulation)
        elif 'shadow' in computer_code.lower():
            sim = asw.read_beam_from_csv(simulation)
        X.append(sim["X"])
        Y.append(sim["Y"])
        dX.append(sim["Xp"])
        dY.append(sim["Yp"])

    abs_X = np.max(np.abs(X))*nmax
    abs_Y = np.max(np.abs(Y))*nmax
    abs_dX = np.max(np.abs(dX))*nmax
    abs_dY = np.max(np.abs(dY))*nmax
    sigma_X = np.std(X)*nsigma
    sigma_Y = np.std(Y)*nsigma
    sigma_dX = np.std(dX)*nsigma
    sigma_dY = np.std(dY)*nsigma

    res_dict = {
        'max': [abs_X, abs_dX, abs_Y, abs_dY],
        'std': [sigma_X, sigma_dX, sigma_Y, sigma_dY]
    }

    return res_dict

def merge_beams(beams):

    merged_beam = {
    'energy': np.array([]),
    'intensity': np.array([]),
    'X': np.array([]),
    'Y': np.array([]),
    'Z': np.array([]),
    'Xp': np.array([]),
    'Yp': np.array([]),
    'Zp': np.array([]),
    }
    for beam in beams:
        for key in merged_beam.keys():
            merged_beam[key] = np.append(merged_beam[key], beam[key])

    return merged_beam

def analise_simulation(simulation, plot_range, mtd='std'):
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(simulation)
    sim = apx.read_beam_from_csv(simulation)
    get_beam_stats(sim)
    plot_phase_space(sim, 'x', prange=[-plot_range[mtd][0]*1E6, plot_range[mtd][0]*1E6, 
                                       -plot_range[mtd][1]*1E6, plot_range[mtd][1]*1E6],  file_name=None)
    plot_phase_space(sim, 'y', prange=[-plot_range[mtd][2]*1E6, plot_range[mtd][2]*1E6, 
                                       -plot_range[mtd][3]*1E6, plot_range[mtd][3]*1E6], file_name=None)
        
    szrange = max(plot_range[mtd][0], plot_range[mtd][2])
    dvrange = max(plot_range[mtd][1], plot_range[mtd][3])
    plot_beam_size(sim, prange=[-szrange*1E6, szrange*1E6, 
                                -szrange*1E6, szrange*1E6], file_name=None)
    plot_beam_div(sim, prange=[-dvrange*1E6, dvrange*1E6, 
                               -dvrange*1E6, dvrange*1E6], file_name=None)

In [20]:
file_list = sorted(glob('./results/optix_up_SII_M1B_G1600_mono_@_mono_foc_ver_E350eV_*'))
# plot_range = get_simulations_range(file_list, 'optix', nsigma=5.)


In [ ]:
print(file_list[0])
mtd='std'
sim = apx.read_beam_from_csv(file_list[0])
get_beam_stats(sim)
plot_phase_space(sim, 'x', prange=[-plot_range[mtd][0]*1E6, plot_range[mtd][0]*1E6, 
                                    -plot_range[mtd][1]*1E6, plot_range[mtd][1]*1E6],  file_name=None)
plot_phase_space(sim, 'y', prange=[-plot_range[mtd][2]*1E6, plot_range[mtd][2]*1E6, 
                                    -plot_range[mtd][3]*1E6, plot_range[mtd][3]*1E6], file_name=None)
        


In [ ]:
file_list = sorted(glob('./results/optix_U52_M1B_G1600_WLT_A_@_cromag_E350eV*'))


In [26]:
# file_list = sorted(glob('./results/mono/optix_U52_M1B_G1600_mono_@_mono_foc_hor_E350eV*'))
file_list = sorted(glob('./results/optix_U52_M1B_G1600_WLT_A_@_cromag_E350eV*'))
plt_range_1 = get_simulations_range(file_list, 'optix', nsigma=5.)

# file_list = sorted(glob('./results/mono/optix_U52_M1B_G1600_mono_@_mono_foc_ver_E350eV*'))
file_list = sorted(glob('./results/optix_U52_M1B_G1600_WLT_A_@_mk2t_E350eV*'))
plt_range_2 = get_simulations_range(file_list, 'optix', nsigma=5.)

In [ ]:
E=1500
# simulation = f'./results/mono/optix_U52_M1B_G1600_mono_@_mono_foc_hor_E{E}eV.csv'
# simulation = f'./results/optix_SII_M1B_G1600_mono_@_mono_foc_hor_E{E}eV.csv'
# simulation = f'./results/optix_U52_M1C_G1600_mono_@_mono_foc_hor_E{E}eV.csv'
# simulation = f'./results/optix_SII_M1C_G1600_mono_@_mono_foc_hor_E{E}eV.csv'
# simulation = f'./results/optix_U52_M1C_G2400_mono_@_mono_foc_hor_E{E}eV.csv'
# simulation = f'./results/optix_SII_M1C_G2400_mono_@_mono_foc_hor_E{E}eV.csv'
# simulation = f'./results/optix_U52_M1B_G1600_WLT_A_@_cromag_E{E}eV.csv'
# simulation = f'./results/optix_SII_M1B_G1600_WLT_A_@_cromag_E{E}eV.csv'
# simulation = f'./results/optix_U52_M1C_G1600_WLT_A_@_cromag_E{E}eV.csv'
# simulation = f'./results/optix_SII_M1C_G1600_WLT_A_@_cromag_E{E}eV.csv'
# simulation = f'./results/optix_U52_M1C_G2400_WLT_A_@_cromag_E{E}eV.csv'
# simulation = f'./results/optix_SII_M1C_G2400_WLT_A_@_cromag_E{E}eV.csv'
# analise_simulation(simulation, plt_range_1)

simulation = f'./results/optix_U52_M1B_G1600_WLT_B_@_cromag_E{E}eV.csv'
simulation = f'./results/optix_SII_M1B_G1600_WLT_B_@_cromag_E{E}eV.csv'
simulation = f'./results/optix_U52_M1C_G1600_WLT_B_@_cromag_E{E}eV.csv'
simulation = f'./results/optix_SII_M1C_G1600_WLT_B_@_cromag_E{E}eV.csv'
simulation = f'./results/optix_U52_M1C_G2400_WLT_B_@_cromag_E{E}eV.csv'
simulation = f'./results/optix_SII_M1C_G2400_WLT_B_@_cromag_E{E}eV.csv'
analise_simulation(simulation, plt_range_2)

# simulation = f'./results/mono/optix_U52_M1B_G1600_mono_@_mono_foc_ver_E{E}eV.csv'
# simulation = f'./results/optix_SII_M1B_G1600_mono_@_mono_foc_ver_E{E}eV.csv'
# simulation = f'./results/optix_U52_M1C_G1600_mono_@_mono_foc_ver_E{E}eV.csv'
# simulation = f'./results/optix_SII_M1C_G1600_mono_@_mono_foc_ver_E{E}eV.csv'
# simulation = f'./results/optix_U52_M1C_G2400_mono_@_mono_foc_ver_E{E}eV.csv'
# simulation = f'./results/optix_SII_M1C_G2400_mono_@_mono_foc_ver_E{E}eV.csv'
# simulation = f'./results/optix_U52_M1B_G1600_WLT_A_@_mk2t_E{E}eV.csv'
# simulation = f'./results/optix_SII_M1B_G1600_WLT_A_@_mk2t_E{E}eV.csv' 
# simulation = f'./results/optix_U52_M1C_G1600_WLT_A_@_mk2t_E{E}eV.csv'
# simulation = f'./results/optix_SII_M1C_G1600_WLT_A_@_mk2t_E{E}eV.csv'
# simulation = f'./results/optix_U52_M1C_G2400_WLT_A_@_mk2t_E{E}eV.csv'
# simulation = f'./results/optix_SII_M1C_G2400_WLT_A_@_mk2t_E{E}eV.csv'
# analise_simulation(simulation, plt_range_2)    

simulation = f'./results/optix_U52_M1B_G1600_WLT_B_@_mk2t_E{E}eV.csv'
simulation = f'./results/optix_SII_M1B_G1600_WLT_B_@_mk2t_E{E}eV.csv' 
simulation = f'./results/optix_U52_M1C_G1600_WLT_B_@_mk2t_E{E}eV.csv'
simulation = f'./results/optix_SII_M1C_G1600_WLT_B_@_mk2t_E{E}eV.csv'
simulation = f'./results/optix_U52_M1C_G2400_WLT_B_@_mk2t_E{E}eV.csv'
simulation = f'./results/optix_SII_M1C_G2400_WLT_B_@_mk2t_E{E}eV.csv'
analise_simulation(simulation, plt_range_1)